### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности. 

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [ ]:
#!pip install transformers[torch] 
#!pip install datasets seqeval
#!pip install corus razdel 
#!pip install torch==2.0.0
#!pip install pandas

In [1]:
# считаем данные
import os 
import torch
import pandas as pd

batch_size = 2
num_train_epochs = 30
model_checkpoint = "xlm-roberta-large"
name_folder_and_model = 'xlm-roberta-large_30.bin'

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
is_cuda = torch.cuda.is_available()
print(is_cuda)
data = pd.read_csv("ner_data_train.csv")

True


In [2]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

In [3]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах
    
    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

### Обработаем датасет и разобьем на трейн и тест

In [4]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [5]:
pd.options.display.max_colwidth = 300

#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [6]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [7]:
from datasets import Dataset, DatasetDict

In [8]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5137
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1285
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [9]:
from transformers import AutoTokenizer 
from datasets import load_dataset, load_metric

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='gpu')

In [10]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'Московский', 'международный', 'фестиваль', 'мира', '=', '89', 'и', 'Стас', 'Намин', '.', '«', 'Главный', 'день', '»', '<', 'ОПИСАНИЕ', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центральном', 'стадионе', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'международный', 'рок', 'фестиваль', '.', 'Концерт', 'собрал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'молодежь', 'тогда', 'увидела', 'и', 'услышала', 'признанных', 'мировых', 'кумиров', ':', 'Scorpions', ',', 'Bon', 'Jovi', ',', 'Ozzy', 'Osbourne', ',', 'Cinderella', ',', 'Motley', 'Crue', '.', 'Приезд', 'западных', 'звезд', 'казался', 'ожившей', 'фантастикой', '.', 'Фестиваль', 'стал', 'не', 'только', 'музыкальной', ',', 'но', 'и', 'важной', 'политической', 'акцией', '.', 'Вся', 'мировая', 'пресса', 'писала', 'о', 'том', ',', 'что', 'СССР', 'и', 'США', ',', 'долгое', 'время', 'находившиеся', 'в', 'состоянии', '"', 'холодной', 'войны', '"', ',', 'наконец', 'то', 'с

In [11]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁<', '▁НА', 'З', 'ВА', 'НИЕ', '▁:', '▁>', '▁Московск', 'ий', '▁между', 'народ', 'ный', '▁фестиваль', '▁мира', '▁=', '▁89', '▁и', '▁Ста', 'с', '▁На', 'мин', '▁', '.', '▁«', '▁Глав', 'ный', '▁день', '▁»', '▁<', '▁О', 'ПИС', 'А', 'НИЕ', '▁:', '▁>', '▁12', '▁и', '▁13', '▁июня', '▁1989', '▁года', '▁на', '▁Центральн', 'ом', '▁стадион', 'е', '▁имени', '▁В', '▁', '.', '▁И', '▁', '.', '▁Ленин', 'а', '▁состоя', 'лся', '▁первый', '▁между', 'народ', 'ный', '▁рок', '▁фестиваль', '▁', '.', '▁Концерт', '▁со', 'брал', '▁свыше', '▁100', '▁тысяч', '▁зрител', 'ей', '▁', '.', '▁Совет', 'ская', '▁молод', 'еж', 'ь', '▁тогда', '▁увидел', 'а', '▁и', '▁услышал', 'а', '▁призна', 'нных', '▁мир', 'овых', '▁ку', 'мир', 'ов', '▁:', '▁S', 'corp', 'ions', '▁', ',', '▁Bon', '▁Jo', 'vi', '▁', ',', '▁O', 'zzy', '▁Os', 'bour', 'ne', '▁', ',', '▁C', 'inde', 'rella', '▁', ',', '▁Mot', 'ley', '▁Cru', 'e', '▁', '.', '▁При', 'езд', '▁запад', 'ных', '▁звезд', '▁каза', 'лся', '▁ожи', 'вшей', '▁фантасти', 'кой', '▁', '.

In [12]:
len(example["tags"]), len(tokenized_input["input_ids"])

(170, 276)

In [13]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 2, 3, 4, 4, 5, 5, 5, 6, 7, 8, 9, 10, 11, 11, 12, 12, 13, 13, 14, 15, 15, 16, 17, 18, 19, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 30, 31, 32, 33, 33, 34, 35, 35, 36, 36, 37, 37, 38, 39, 39, 39, 40, 41, 42, 42, 43, 44, 44, 45, 46, 47, 48, 48, 49, 49, 50, 50, 51, 51, 51, 52, 53, 53, 54, 55, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 61, 62, 63, 63, 64, 64, 65, 65, 66, 66, 66, 67, 67, 68, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 73, 73, 74, 74, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 82, 83, 84, 84, 85, 85, 86, 87, 88, 88, 89, 90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 97, 98, 98, 99, 100, 101, 102, 103, 103, 104, 104, 105, 106, 106, 107, 108, 109, 110, 110, 111, 112, 113, 113, 114, 115, 116, 117, 117, 117, 118, 118, 118, 118, 119, 120, 121, 121, 122, 122, 123, 123, 124, 124, 125, 125, 126, 126, 127, 127, 128, 128, 128, 129, 130, 130, 131, 132, 132, 133, 133, 134, 134, 135, 136, 137, 137, 138, 139, 139, 140, 141, 141, 142, 

In [14]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

276 276


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [15]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[0, 4426, 5315, 4026, 29889, 189182, 152, 977, 51208, 75586, 54039, 6, 83389, 45799, 1270, 102356, 4426, 1089, 157933, 1709, 189182, 152, 977, 123811, 336, 10698, 4558, 8568, 111529, 89, 69361, 19776, 100347, 292, 51208, 75586, 54039, 6, 83389, 45799, 1270, 102356, 6, 5, 417, 3920, 8165, 47745, 90216, 10041, 13549, 4476, 6, 5, 1089, 25358, 98590, 35, 98465, 100347, 1214, 129, 49571, 546, 135, 170769, 146408, 131567, 6, 47871, 1435, 8113, 1560, 8568, 129, 184862, 312, 1089, 88691, 89, 93362, 419, 93834, 17968, 63781, 6, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 2

In [17]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [20]:
metric = load_metric("seqeval")

C:\Users\tommy\AppData\Local\Temp\ipykernel_19660\152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [21]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'бренд': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [22]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [23]:
# что мы видим без дообучения модели
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.375157356262207,
 'eval_precision': 0.004121586810922205,
 'eval_recall': 0.0381474949376184,
 'eval_f1': 0.007439395676488197,
 'eval_accuracy': 0.009479054211384178,
 'eval_runtime': 35.7511,
 'eval_samples_per_second': 35.943,
 'eval_steps_per_second': 17.985}

In [24]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [25]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными 
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [29]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'max_split_size_mb'

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 8.00 GiB total capacity; 7.17 GiB already allocated; 0 bytes free; 7.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.evaluate()

In [ ]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

In [ ]:
model.save_pretrained(name_folder_and_model)
tokenizer.save_pretrained(name_folder_and_model)

### Посмотрим на результаты

In [ ]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    
    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:    
            print(f'{t:15s} {label_list[idx]:10s}')
    return text, pipe(text), labels

In [ ]:
predict_ner(text, tokenizer, model, pipe)

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [ ]:
#from tqdm.notebook import tqdm

submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(text, tokenizer, model, pipe, submission):
    for i, elem in enumerate(ner_test):
        _, _, labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
        submission.loc[i, 'video_info'] = elem

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [ ]:
result = sample_submission(text, tokenizer, model, pipe, submission)

In [ ]:
result

In [ ]:
len(ner_test)

In [ ]:
result.to_csv('test.csv')